# Classification based machine learning for trading in R

In [19]:
install.packages("lubridate")
library(lubridate)
library(quantmod)


The downloaded binary packages are in
	/var/folders/rh/pfd6hrw52s35skb7d44l36nr0000gn/T//RtmpkYsINl/downloaded_packages


In [25]:
# Load the data
price <- read.csv("data/wtiDaily.csv")
head(price)

,X7.13.2016,X53.48,X53.61,X51.46,X51.65
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,7/14/2016,52.04,52.70,51.85,52.58
2,7/15/2016,52.42,53.23,51.95,52.85
3,7/18/2016,53.03,53.04,51.74,52.14
4,7/19/2016,52.13,52.58,51.48,51.65
5,7/20/2016,51.62,52.33,50.75,51.95
6,7/21/2016,51.85,52.29,50.72,50.95


In [26]:
setup_data <- function(pricedata) {
    names(pricedata) <- c("Date", "Open", "High", "Low", "Close")
    dates <- parse_date_time(x = pricedata[,1], "mdy_HM", truncated = 3)
    pricedata <- pricedata[,2:5]
    pricedata <- xts(pricedata, order.by = dates)
}

In [27]:
price <- setup_data(price)

In [30]:
head(price)

Warning message:
“timezone of object (UTC) is different than current timezone ().”


            Open  High   Low Close
2016-07-14 52.04 52.70 51.85 52.58
2016-07-15 52.42 53.23 51.95 52.85
2016-07-18 53.03 53.04 51.74 52.14
2016-07-19 52.13 52.58 51.48 51.65
2016-07-20 51.62 52.33 50.75 51.95
2016-07-21 51.85 52.29 50.72 50.95

In [31]:
summary(price)

     Index                          Open            High            Low       
 Min.   :2016-07-14 00:00:00   Min.   :42.66   Min.   :43.31   Min.   :42.16  
 1st Qu.:2017-01-20 00:00:00   1st Qu.:49.77   1st Qu.:50.40   1st Qu.:49.06  
 Median :2017-07-31 00:00:00   Median :54.55   Median :55.42   Median :54.01  
 Mean   :2017-07-30 05:24:28   Mean   :55.66   Mean   :56.34   Mean   :54.95  
 3rd Qu.:2018-02-07 00:00:00   3rd Qu.:61.15   3rd Qu.:61.84   3rd Qu.:60.44  
 Max.   :2018-08-15 00:00:00   Max.   :73.45   Max.   :74.37   Max.   :72.75  
     Close      
 Min.   :42.64  
 1st Qu.:49.77  
 Median :54.73  
 Mean   :55.68  
 3rd Qu.:61.12  
 Max.   :73.43  

In [32]:
ema7 <- EMA(price$Close, n= 7)  # Exponential Moving Average
ema20 <- EMA(price$Close, n=20) # Exponential Moving Average

In [35]:
tail(ema7)

Warning message:
“timezone of object (UTC) is different than current timezone ().”


                EMA
2018-08-08 68.39387
2018-08-09 67.99790
2018-08-10 67.90593
2018-08-13 67.72944
2018-08-14 67.55708
2018-08-15 66.86031

In [36]:
tail(ema20)

Warning message:
“timezone of object (UTC) is different than current timezone ().”


                EMA
2018-08-08 68.79087
2018-08-09 68.60222
2018-08-10 68.50963
2018-08-13 68.38490
2018-08-14 68.25681
2018-08-15 67.92474